# **1. Setting**

In [1]:
import cv2
import torch
from google.colab.patches import cv2_imshow
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/Jaeyoon-Park/project01
%cd project01

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 97 not upgraded.
Need to get 6,800 kB of archives.
After this operation, 15.3 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.1.2 [6,800 kB]
Fetched 6,800 kB in 1s (13.4 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

미리 custom data로 학습된 가중치 파일 best.pt 로드하기

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-4-11 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


비디오 정보 확인

In [3]:
capture = cv2.VideoCapture("lostark sample.mp4")

capture_w = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
capture_h = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
capture_total_frame = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

fps = capture.get(cv2.CAP_PROP_FPS)

print("{0} x {1}".format(capture_w, capture_h))
print(capture_total_frame)
print(fps)

1920 x 1080
8407
29.978643737677537


스킬을 확인할 수 있는 비디오 프레임 확인 및 배경으로 설정

In [4]:
# 슬라이더로 프레임 설정
import ipywidgets as widgets
slider = widgets.IntSlider(value=5, max=capture_total_frame)
display(slider)

IntSlider(value=5, max=8407)

In [5]:
# 슬라이더 값으로 비디오 프레임 조정 및 화면 확인
capture.set(cv2.CAP_PROP_POS_FRAMES, slider.value)
_, bg = capture.read()
cv2_imshow(bg)

Output hidden; open in https://colab.research.google.com to view.

헤드어택 성공 비율을 확인하려는 스킬 선택

In [6]:
list_skill_alpha = ["Q", "W", "E", "R",
                    "A", "S", "D", "F"]

list_skill = [(682, 978, 45, 45), (729, 978, 45, 45), (776, 978, 45, 45), (823, 815, 45, 45),
              (704, 1025, 45, 45), (751, 1025, 45, 45), (798, 1025, 45, 45), (845, 1025, 45, 45)]

# 확인하고자 하는 스킬 입력
input_skill = input("확인하려는 스킬은? ")

sk_idx = 0
while input_skill != list_skill_alpha[sk_idx]:
    sk_idx = sk_idx+1

roi_x1 = int(list_skill[sk_idx][0])
roi_x2 = int(list_skill[sk_idx][0]+list_skill[sk_idx][2])

roi_y1 = int(list_skill[sk_idx][1])
roi_y2 = int(list_skill[sk_idx][1]+list_skill[sk_idx][3])

img_bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)
roi_bg = img_bg[roi_y1:roi_y2, roi_x1:roi_x2] # (y, x)
cut_bg = img_bg[0:25, 110:333]

확인하려는 스킬은? A


# **2. Processing**

해당 스킬 사용 횟수 및 프레임 확인

In [7]:
# 프레임 초기화
capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

skill_use_frame_mean_list = [0]
cutscene_mean_list = [0]
skill_use_frame_idx_list = []

for i in range(1, int(capture_total_frame)):
    run, frame = capture.read()
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = img[roi_y1:roi_y2, roi_x1:roi_x2] # (y, x)
    cut = img[0:25, 110:333] # 좌측 상단 메뉴바

    skill_use = cv2.absdiff(roi, roi_bg)
    mean_value_skill_use = float(cv2.mean(skill_use)[0])
    skill_use_frame_mean_list.append(mean_value_skill_use)
    skill_used = abs(skill_use_frame_mean_list[i-1]-skill_use_frame_mean_list[i])

    # 플레이 중 컷신 발생에 대한 예외 처리 (상단 메뉴바 이용)
    cutscene_diff = cv2.absdiff(cut, cut_bg)
    cutscene_chk = float(cv2.mean(cutscene_diff)[0])
    cutscene_mean_list.append(cutscene_chk)
    cutscene = abs(cutscene_mean_list[i-1]-cutscene_mean_list[i])
    
    # 스킬창 roi 변화량과 상단 메뉴바 roi 변화량 기준으로 스킬 사용 여부 체크
    if skill_used > 70 and cutscene < 30:
      skill_use_frame_idx_list.append(int(capture.get(cv2.CAP_PROP_POS_FRAMES)))
      cv2_imshow(frame)

Output hidden; open in https://colab.research.google.com to view.

스킬 사용 당시 헤드어택 발생 여부 확인 (custom YOLOv5 모델 적용)

In [8]:
headcnt = 0
model.conf = 0.9
for i in range(len(skill_use_frame_idx_list)):
    capture.set(cv2.CAP_PROP_POS_FRAMES, int(skill_use_frame_idx_list[i])+10)
    while True:
      run2, frame2 = capture.read()

      if i < len(skill_use_frame_idx_list)-1:
        if skill_use_frame_idx_list[i+1] - skill_use_frame_idx_list[i] > 60:
          magic_num = 60
        else:
          magic_num = int((skill_use_frame_idx_list[i+1] - skill_use_frame_idx_list[i])/2)

      if capture.get(cv2.CAP_PROP_POS_FRAMES) >= int(skill_use_frame_idx_list[i]+magic_num):
        break
      if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture_total_frame:
        break

      img2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
      area1 = img2[220:860, 640:1280]
      area2 = img2[60:700, 640:1280]

      detecting1 = model(area1)
      if len(detecting1.pandas().xyxy[0])>0:
        headcnt = headcnt+1
        detecting1.save()
        cv2_imshow(frame2)
        print(capture.get(cv2.CAP_PROP_POS_FRAMES))
        break
      else:
        detecting2 = model(area2)
        if len(detecting2.pandas().xyxy[0])>0:
          headcnt = headcnt+1
          detecting2.save()
          cv2_imshow(frame2)
          print(capture.get(cv2.CAP_PROP_POS_FRAMES))
          break

Output hidden; open in https://colab.research.google.com to view.

# **3. Results**

In [10]:
print(input_skill+" 스킬의 헤드어택 성공 비율은 " + format(headcnt/len(skill_use_frame_idx_list)*100, ".2f")+"%입니다")
print("총 스킬 사용횟수 : " + str(len(skill_use_frame_idx_list)))
print("헤드어택 횟수 : " + str(headcnt))

A 스킬의 헤드어택 성공 비율은 81.82%입니다
총 스킬 사용횟수 : 11
헤드어택 횟수 : 9
